In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import Dataset
import pytorch_lightning as pl

pl.seed_everything(0)

Global seed set to 0


0

In [3]:
class MyDataset(Dataset):
  def __init__(self, size=5000, dim=40, random_offset=0):
        super(MyDataset, self).__init__()
        self.size = size
        self.dim = dim
        self.random_offset = random_offset

  def __getitem__(self, index):
      if index >= len(self):
          raise IndexError("{} index out of range".format(self.__class__.__name__))

      rng_state = torch.get_rng_state()
      torch.manual_seed(index + self.random_offset)

      while True:
        img = torch.zeros(self.dim, self.dim)
        dx = torch.randint(-10,10,(1,),dtype=torch.float)
        dy = torch.randint(-10,10,(1,),dtype=torch.float)
        c = torch.randint(-20,20,(1,), dtype=torch.float)

        params = torch.cat((dy/dx, c))
        xy = torch.randint(0,img.shape[1], (20, 2), dtype=torch.float)
        xy[:,1] = xy[:,0] * params[0] + params[1]

        xy.round_()
        xy = xy[ xy[:,1] > 0 ]
        xy = xy[ xy[:,1] < self.dim ]
        xy = xy[ xy[:,0] < self.dim ]

        for i in range(xy.shape[0]):
          x, y = xy[i][0], self.dim - xy[i][1]
          img[int(y), int(x)]=1
        if img.sum() > 2:
          break

      torch.set_rng_state(rng_state)
      return img.unsqueeze(0), params

  def __len__(self):
      return self.size

train_data = MyDataset()
val_data = MyDataset(size=500, random_offset=33333)
test_data = MyDataset(size=500, random_offset=99999)

## Task 1.1

In [4]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [5]:
trainloader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=4)
valloader = DataLoader(val_data, batch_size=32, num_workers=4)
testloader = DataLoader(test_data, batch_size=128, num_workers=4)

In [6]:
class BaselineCNN(pl.LightningModule):
    
    def __init__(self): # input: 40 x 40
        super().__init__()

        self.conv1 = nn.Conv2d(1, 48, (3, 3), stride=1, padding=1)
        self.fc1 = nn.Linear(48 * 40 * 40, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        batch_size, channels, width, height = x.size()

        x = self.conv1(x) # output: 48 x 40 x 40
        x = F.relu(x)
        x = x.view(batch_size, -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y, y_hat)

        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y, y_hat)

        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)


    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y, y_hat)

        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())



In [7]:
model = BaselineCNN().to('cuda:1')

trainer = pl.Trainer(gpus=1, max_epochs=100, deterministic=True, auto_select_gpus=True)

trainer.fit(model, trainloader, valloader)

   | 0/16 [00:00<?, ?it/s]
Epoch 54:  71%|███████▏  | 40/56 [00:02<00:00, 18.51it/s, loss=0.618, v_num=15, val_loss=9.530, train_loss=0.664]
Validating: 0it [00:00, ?it/s]
Epoch 55:  71%|███████▏  | 40/56 [00:02<00:00, 19.13it/s, loss=0.516, v_num=15, val_loss=9.450, train_loss=0.617]
Validating: 0it [00:00, ?it/s]
Epoch 56:  71%|███████▏  | 40/56 [00:02<00:00, 18.19it/s, loss=0.527, v_num=15, val_loss=9.470, train_loss=0.542]
Validating: 0it [00:00, ?it/s]
Epoch 57:  71%|███████▏  | 40/56 [00:02<00:00, 16.37it/s, loss=0.563, v_num=15, val_loss=9.340, train_loss=0.514]
Validating: 0it [00:00, ?it/s]
Epoch 58:  71%|███████▏  | 40/56 [00:02<00:00, 17.16it/s, loss=0.486, v_num=15, val_loss=9.520, train_loss=0.489]
Validating: 0it [00:00, ?it/s]
Epoch 59:  71%|███████▏  | 40/56 [00:02<00:00, 17.49it/s, loss=0.518, v_num=15, val_loss=9.450, train_loss=0.511]
Validating: 0it [00:00, ?it/s]
Epoch 60:  71%|███████▏  | 40/56 [00:02<00:00, 17.45it/s, loss=0.478, v_num=15, val_loss=9.380, train_l

1